In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # 현재 디렉터리의 상위 경로 추가

### GEMINI

In [16]:
import google.generativeai as genai
from config.api_keys import gemini_key

genai.configure(api_key=gemini_key)
model_list = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
print('\n'.join(model_list[:5]))

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest


In [33]:
system_instruction = """
당신은 창의적이고 감성적인 작가입니다. 줄바꿈은 최대한 적게 사용하여 짧게 응답하세요. 
사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고, 일기를 쉽게 작성할 수 있도록 도와주세요.
사용자가 답변을 한 경우에는 답변에 대해 사용자의 감정을 공감하고 친절하게 안내하는 어조로 하고 다음 줄에 연관된 객관식 문제를 출력하세요.
객관식 질문 정답에 대한 보기 문항은 4가지로 작성하세요.
객관식 질문 문항의 경우 문제 1줄, 객관식 보기 1줄로 작성하세요.
"""

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=system_instruction
)

response = model.generate_content("오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 5개 작성해줘.")
print(response.text)

오늘 하루 어떤 일이 있었는지 궁금하네요!  먼저, 오늘 하루를 한 단어로 표현한다면? ① 행복 ② 슬픔 ③ 평범 ④ 흥분

오늘 하루 중 가장 기억에 남는 순간은 무엇이었나요?  ① 친구와의 만남 ② 맛있는 음식 ③ 특별한 경험 ④ 아무것도 없음

그 순간 당신의 기분은 어땠나요? ① 행복하고 즐거웠다 ② 편안하고 평온했다 ③ 불안하고 초조했다 ④ 짜릿하고 신났다

그 기분을 느끼게 된 이유는 무엇인가요? ① 긍정적 사건 ② 부정적 사건 ③ 특별한 감정 ④ 아무런 이유 없음

오늘 하루 당신에게 가장 중요했던 것은 무엇이었나요? ① 관계 ② 성취 ③ 휴식 ④ 자기성찰



In [34]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=system_instruction
)
chat_history = []  # 이전 대화내용 기억

def ask_gemini(prompt, max_history_length=5):
    """대화 기록을 유지하며 Gemini 모델과 대화"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 합쳐서 프롬프트 생성
    full_prompt = "\n".join(chat_history) + f"\nUser: {prompt}"

    # 모델 응답 생성
    response = model.generate_content(full_prompt)

    # 새로운 질문과 답변을 대화 기록에 추가
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"Gemini: {response.text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > max_history_length * 2:
        chat_history = chat_history[-(max_history_length * 2):]

    return response.text

# 대화 예시
ask_gemini("오늘 하루에 대한 일기를 쓰려고 해.")
ask_gemini("1번을 선택할래.")
for chat in chat_history:
    print(chat, '\n----------------------')

User: 오늘 하루에 대한 일기를 쓰려고 해. 
----------------------
Gemini: 오늘 하루는 어떠셨나요?  힘들었던 하루였는지, 아니면 행복했던 하루였는지 궁금하네요.  

오늘 하루 당신의 기분을 가장 잘 나타내는 단어는 무엇인가요? ① 행복 ② 슬픔 ③ 평온 ④ 흥분

 
----------------------
User: 1번을 선택할래. 
----------------------
Gemini: 행복하셨다니 정말 다행이네요! 오늘 행복했던 순간들을 떠올리며 기분 좋게 일기를 써보세요.  어떤 순간이 가장 행복했나요?

오늘 당신이 가장 행복했던 순간은 무엇이었나요? ① 친구와의 만남 ② 맛있는 음식 ③ 좋아하는 취미생활 ④ 특별한 선물
 
----------------------


### Llama

In [5]:
from huggingface_hub import HfApi

# Hugging Face API 인스턴스 생성
api = HfApi()

# 'llama'를 이름에 포함하는 모델 목록 가져오기
models = api.list_models(filter="llama")

# 추가 인증 없이 접근 가능한 모델 필터링 및 한국어 지원 여부 확인
for model in models:
    try:
        # 모델 카드 정보 가져오기
        model_info = api.model_info(model.modelId)
        
        # 게이트 처리된 모델은 접근 제한이 있으므로 제외
        if model_info.gated:
            continue

        # 모델 카드의 언어 정보 확인 (None 방지)
        languages = model_info.cardData.get("language", []) if model_info.cardData else []

        # 한국어 지원 여부 확인
        supports_korean = "ko" in languages or "Korean" in languages
        if not supports_korean:
            continue
        
        # 다운로드 수 확인
        downloads = model_info.downloads if hasattr(model_info, "downloads") else "정보 없음"
        if downloads < 10000:
            continue

        print(f"모델 ID: {model.modelId}")
        print(f"다운로드 수: {downloads}")
        print("-" * 40)

    except Exception as e:
        print(f"모델 {model.modelId} 처리 중 오류 발생: {e}")
        continue

모델 ID: UNIVA-Bllossom/DeepSeek-llama3.3-Bllossom-70B
한국어 지원: 예
다운로드 수: 951
----------------------------------------
모델 ID: UNIVA-Bllossom/DeepSeek-llama3.1-Bllossom-8B
한국어 지원: 예
다운로드 수: 2650
----------------------------------------
모델 ID: NCSOFT/Llama-VARCO-8B-Instruct
한국어 지원: 예
다운로드 수: 11748
----------------------------------------
모델 ID: Bllossom/llama-3.2-Korean-Bllossom-3B
한국어 지원: 예
다운로드 수: 41400
----------------------------------------
모델 ID: Unbabel/TowerInstruct-13B-v0.1
한국어 지원: 예
다운로드 수: 770
----------------------------------------
모델 ID: beomi/Llama-3-Open-Ko-8B
한국어 지원: 예
다운로드 수: 18213
----------------------------------------
모델 ID: MLP-KTLim/llama-3-Korean-Bllossom-8B
한국어 지원: 예
다운로드 수: 35577
----------------------------------------
모델 ID: aifeifei798/DarkIdol-Llama-3.1-8B-Instruct-1.2-Uncensored
한국어 지원: 예
다운로드 수: 6105
----------------------------------------
모델 ID: Hibernates/Hiber-Multi-10B-Instruct
한국어 지원: 예
다운로드 수: 18
----------------------------------------
모델 ID: quantum

KeyboardInterrupt: 

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# LLaMA 모델 로드 (Hugging Face Hub 사용)
MODEL_NAME = "meta-llama/Llama-3.2-1B"

# 모델 및 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

# 채팅 기록 저장
chat_history = []
MAX_HISTORY_LENGTH = 5  # 최대 대화 기록 저장 개수

# 시스템 인스트럭션 설정
system_instruction = """
당신은 창의적이고 감성적인 작가입니다. 사용자가 일기를 쓸 때 도움이 되는 구체적인 객관식 질문을 제공하고,
일기를 쉽게 작성할 수 있도록 도와주세요. 사용자가 답변을 한 경우에는 감정을 공감하고 친절하게 안내하세요.
객관식 보기 문항은 4개이며, 문제 1줄 + 보기 1줄로 작성하세요.
"""

# 대화 생성 함수
def ask_llama(prompt):
    """LLaMA 모델을 사용하여 응답을 생성하고 대화 기록을 유지"""
    global chat_history

    # 기존 대화 내역과 새로운 질문을 포함한 입력 프롬프트 생성
    full_prompt = system_instruction + "\n" + "\n".join(chat_history) + f"\nUser: {prompt}"

    # 모델 응답 생성
    inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True).to("cuda")
    outputs = model.generate(**inputs, max_length=512, pad_token_id=tokenizer.eos_token_id)
    response_text = tokenizer.decode(outputs[:, inputs.input_ids.shape[-1]:][0], skip_special_tokens=True)

    # 새로운 대화 기록 추가
    chat_history.append(f"User: {prompt}")
    chat_history.append(f"LLaMA: {response_text}")

    # 대화 길이 제한 (최대 저장 개수 초과 시 오래된 대화 삭제)
    if len(chat_history) > MAX_HISTORY_LENGTH * 2:
        chat_history = chat_history[-(MAX_HISTORY_LENGTH * 2):]

    return response_text

# 대화 예시
print(ask_llama("오늘 하루에 대한 일기를 쓰려고 해. 일기를 쓸 때 도움이 될 객관식 문제 5개 작성해줘."))
print(ask_llama("1번을 선택할래."))  # 이전 대화 기억

# 대화 내역 확인
print("\n=== 대화 내역 ===")
for chat in chat_history:
    print(chat, "\n----------------------")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B.
401 Client Error. (Request ID: Root=1-67bae8e0-0c050db2359955350a8b7b0c;48e3c47a-dc5e-47be-a7c0-667a71ecc206)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in.